# Convolutional Auto-encoder and Dimension Reduction


![index.jpg](attachment:index.jpg)

This workshop deals with a new neural network architecture: auto-encoders. This architecture consists of two parts, the encoder and the decoder. It enables us to build a new, more compact representation of a dataset, with fewer descriptors, thus reducing the dataset's dimensionality. In this workshop, we'll take a look at how auto-encoders work on a classification problem.

# Dataset import and loading

The dataset used in this Workshop is the famous MNIST dataset (handwritten digits). We use the CSV version of this dataset, available here: https://pjreddie.com/projects/mnist-in-csv/

We'll keep only the mnist_test.csv file, containing 10,000 grayscale images of size 28 x 28.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df = pd.read_csv(r'mnist_test.csv', header=None)
df['pixels'] = df.index.map(lambda x: np.array(df.iloc[x][1:]))
dropcols = df.columns[(df.columns != 0) * (df.columns != 'pixels')]
df.drop(dropcols, axis=1, inplace=True)
df.columns = ['label','pixels']
print(df.shape)

In [ ]:
df.head()

We can start by displaying an example image from the MNIST dataset (Grayscale ==> Image)

In [ ]:
fig = plt.imshow(df['pixels'][10].reshape(28,28), cmap='gray')

# Size reduction

In this section, we'll look at how to distinguish between the features of different handwritten digits, using only the raw pixel values as features.

Let's look at 3 numbers. Keep them as they are for now, and you'll have time to play around with them in the end.

In [ ]:
## Figures considered ##
labels = [1,6,8]
colors = ['red', 'blue', 'green']

X = np.array([df['pixels'][i] for i in df.index if df['label'][i] in labels])
y = #PLEASE COMPLETE

print('X shape: '+str(X.shape))
print('y shape: '+str(y.shape))

We have 28 x 28 = 784 features. This is a high dimension (~ same order of magnitude as the number of points).

In this case, we can't rely on univariate analysis: it's obvious that the value of a given pixel in the image won't, on its own, determine the number that is written (the class). We need to study the relationship between pixel values: let's see how dimensionality reduction methods can help us distinguish noise, correlation and information patterns!

### Principal Component Analysis (PCA): Linear Approach

As you saw last year in the IA block (ML Pipeline section), we can use PCA (Principal Component Analysis) to analyze and reduce image size. For more details on PCA, please consult the following links:
* https://ocw.mit.edu/courses/mathematics/18-650-statistics-for-applications-fall-2016/lecture-videos/lecture-19-video/
* https://ocw.mit.edu/courses/mathematics/18-650-statistics-for-applications-fall-2016/lecture-slides/MIT18_650F16_PCA.pdf
* https://ocw.mit.edu/courses/brain-and-cognitive-sciences/9-641j-introduction-to-neural-networks-spring-2005/lecture-notes/lec18_pca.pdf

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
## PCA calculation
pca = #PLEASE COMPLETE
XPCA = #PLEASE COMPLETE


In [ ]:
XPCA.shape

In [ ]:
## Axis selection for visualization
component_x = 1
component_y = 2

## Plotting
fig, ax = plt.subplots()
for i in range(len(labels)):
    ax.scatter(XPCA[y == labels[i],component_x-1],
               XPCA[y == labels[i],component_y-1],
               c=colors[i], label=labels[i], alpha=0.4)

l = ax.legend()

It's also possible to display the data in 3D to show an extra dimension of our PCA-generated vector.

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
## Choice of axes for visualization
component_x = 1
component_y = 2
component_z = 3

## Plotting
%matplotlib inline
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
for i in range(len(labels)):
    ax.scatter(XPCA[y == labels[i],  component_x-1],
               XPCA[y == labels[i],component_y-1],
               XPCA[y == labels[i],component_z-1],
               c=colors[i], label=labels[i], alpha=0.4)

l = ax.legend()

Try to visualize different axes and find those that do (or don't) distinguish classes.

### T-distributed Stochastic Neighbor Embedding : Approche non-Linéaire

Most data sets have a non-linear relationship between features and data points in a high-dimensional space. Therefore, we want to create a low-dimensional encoding for these data that preserves the relationship between the different points in the original space. To this end, the TSNE (T-distributed Stochastic Neighbor Embedding) algorithm is a probabilistic approach that places objects (e.g. images), described by high-dimensional vectors (e.g. grayscale values), in a low-dimensional space in such a way as to preserve the identity of the neighbors. For more details on this algorithm, please consult the following resources:
* http://www.cs.columbia.edu/~verma/classes/uml/lec/uml_lec8_tsne.pdf
* https://www.cs.toronto.edu/~jlucas/teaching/csc411/lectures/lec13_handout.pdf
    

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
## Calculation of t-SNE 2D projection

## Parameters with a real influence on accuracy
perplexity = 30
learning_rate = 200
n_iter = 1000

tsne = TSNE(n_components=2, perplexity=perplexity, learning_rate=learning_rate, n_iter=n_iter)
XTSNE = #PLEASE COMPLETE


In [ ]:
## Plotting
fig, ax = plt.subplots()
for i in range(len(labels)):
    ax.scatter(XTSNE[y == labels[i],0],
               XTSNE[y == labels[i],1],
               c=colors[i], label=labels[i], alpha=0.4)
l = ax.legend()

# Impact on supervised model performance

In this section, we consider a supervised machine learning model for predicting the class of an image (which digit it corresponds to). We assume that the classification task, as well as the algorithms for solving it, are skills acquired in the AI Block. For more information on this section, please refer to the Classification and ML Pipeline sections covered in 4th year.


We will study the impact of dimensionality reduction on 3 classifiers with different mechanisms:
* Naive Bayes Classifier (NB)
* SVM (SVM)
* Random Forest Classifier (RF)

We can give them 3 different inputs as characteristics:

* All pixel values (raw)
* The first n components of the PCA (10 for example)
* The 2 dimensions of the t-SNE projection (tsne)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [ ]:
def fit_my_model(model, features, test_size):

    ## Building train and test sets from all the  features
    if features == 'raw':
        X_train, X_test, y_train, y_test = #PLEASE COMPLETE
    elif features == 'tsne':
        X_train, X_test, y_train, y_test = #PLEASE COMPLETE
    else:
        X_train, X_test, y_train, y_test = #PLEASE COMPLETE

    print("Training samples: "+str(X_train.shape[0]))
    print("Testing samples: "+str(X_test.shape[0]))
    print("Number of features: "+str(X_train.shape[1]))

    ## Fit model
    if model == 'nb':
        clf = GaussianNB()
    elif model == 'svm':
        clf = SVC(gamma='auto')
    elif model == 'rf':
        clf = RandomForestClassifier(n_estimators=200, criterion='gini', max_depth=None, max_features=np.min([10, X_train.shape[1]]))

    clf.fit(X_train, y_train)

    ## Print scores
    learningScore = clf.score(X_train, y_train)
    generalizationScore = clf.score(X_test, y_test)
    print('Learning score: '+str(learningScore))
    print('Generalization score: '+str(generalizationScore))

    return generalizationScore

Now we choose the classification model and the dimension reduction method. The reduction will be the input to the classification model.

In [ ]:
model = 'rf' ## svm, nb, rf
features = 'raw' ## raw, tsne, or integer corresponding to the first n PCA components
test_size = 0.2

score = fit_my_model(model, features, test_size)

Let's test all configurations and build a results table with test scores, to help you answer this question.

In [ ]:
comparison_table = pd.DataFrame(columns = ['raw', 'pca', 'tsne'], index=['svm', 'nb', 'rf'])
for f in comparison_table.columns:
    for m in comparison_table.index:
        print("\n"+f + ' - ' + m)
        comparison_table.loc[m,f] = fit_my_model(m, f if f!='pca' else 10, 0.2)

Let's visualize the result as a heat map.

In [ ]:
## Heatmap with summarized results
fig = sns.heatmap(comparison_table.astype('float'), annot=True, cmap='Reds')

What is the effect of dimensionality reduction on the 3 classifiers? Try to explain why with your intuition...
<em>PLEASE COMPLETE</em>

These examples only concern supervised classifiers. Can you imagine the impact on other tasks: regression, clustering, anomaly detection...?
<em>PLEASE COMPLETE</em>

# Auto-encoders

In this section, we will build a step-by-step autoencoder architecture, train it and evaluate it on MNIST data. We'll start with a standard, fully-connected autoencoder, followed by a variational autoencoder in the next section.

To prepare for this work, let's import the appropriate libraries and use the full Keras MNIST dataset, as neural networks will need more data for training.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [ ]:
import numpy as np

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train = np.array([x.flatten() for x in X_train]).astype("float32") / 255.
X_test = np.array([x.flatten() for x in X_test]).astype("float32") / 255.
print(X_train.shape)
print(X_test.shape)

It is customary to normalize the input data of a neural network between 0 and 1, as this speeds up the training of the network.

### Standard auto-encoder

In this first part, we implement a standard (fully connected) auto-encoder. Let's start with the separate encoder and decoder architectures. Pay attention to the activation of the final layer!

In this section, we will build an Encoder including:
* A 28x28 input data layer,
* A Fully connected hidden layer of 200 units,
* an output of 15 units (you can change the number), which represents the size of the latent space.

![autoencoder-architecture.png](attachment:autoencoder-architecture.png)

In [ ]:
latent_space_dim = #PLEASE COMPLETE

# Encoder architecture
encoder_inputs = #PLEASE COMPLETE
hidden1 = #PLEASE COMPLETE
latent_space = #PLEASE COMPLETE

encoder = keras.Model(encoder_inputs, latent_space, name="encoder")


You can even display and save the model structure:

In [ ]:
from keras.utils import plot_model
plot_model(encoder, to_file='encodersimple.png', show_shapes=True)

Now let's turn to the decoder. Applying the same principle, this cell is symmetrical to the previous one. The only difference is the last layer, where the activation function will use sigmoid.

In [ ]:
# Decoder architecture
decoder_inputs = #PLEASE COMPLETE
hidden2 = #PLEASE COMPLETE
decoder_outputs = #PLEASE COMPLETE

decoder = keras.Model(decoder_inputs, decoder_outputs, name="decoder")

Here too, the model is displayed and saved.

In [ ]:
from keras.utils import plot_model
plot_model(decoder, to_file='decodersimple.png', show_shapes=True)

Now we'll combine the two architectures (Encoder, Decoder) to build the complete AutoEncoder (AE). As a reminder, the encoder output is the decoder input.

In [ ]:
# Combining the two Architecture (Enc,Dec)
outputs = #PLEASE COMPLETE
autoencoder = #PLEASE COMPLETE


In [ ]:
autoencoder.summary()

# Note: it is possible to create an Autoencoder class to create a reusable architecture.  
See https://www.tensorflow.org/tutorials/generative/autoencoder#first_example_basic_autoencoder

We now need to define the loss function for training. As this is a classification task, and as you saw last year (Block IA - Classification), in the case of MNIST images, we can use a binary cross-entropy per pixel, summed over the whole image. In other words, we can use the binary cross-entropy function to estimate the reconstruction error on each pixel of the image. In total, we'll have 784 cross-entropy values for which we'll calculate an average. This average represents the total error. We can also use a root-mean-square error, but let's leave that as an exercise.

In [ ]:
# Definition of the loss function (Loss Fct)
reconstruction_loss = keras.losses.binary_crossentropy(encoder_inputs, outputs) * 784

autoencoder_loss = K.mean(reconstruction_loss)

All that remains is to add the loss function to the auto-encoder, and compile the model.

In [ ]:
# Compiling the model
autoencoder.add_loss(autoencoder_loss)
autoencoder.compile(optimizer="adam")

Let's see what happens. Let's train the model over 30 epochs, dividing the dataset into batches of 128 :

In [ ]:
# Executing the model
history = autoencoder.fit(X_train, X_train,
          #PLEASE COMPLETE


We'll save the weights for future use.

In [ ]:
autoencoder.save_weights('./autoencoder-77.5.h5')


To get an idea of the performance of the model we've developed, we'll visualize the values of the Loss function.

In [ ]:
# Visualization of learning (Train) and validation (Test) losses
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

We seem to be getting a pretty reasonable model! Let's have a look at the quality of the reconstructed images.

In [ ]:
n = 10
plt.figure(figsize=(20, 4))

for i in range(n):
    # Original images
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28), cmap="gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Reconstructed images
    ax = plt.subplot(2, n, i + 1 + n)
    encoded_img = encoder.predict(np.array([X_test[i]]).reshape(1,784))
    decoded_img = decoder.predict(encoded_img).reshape(28, 28)
    plt.imshow(decoded_img, cmap="gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

We can see that some details are lost, but the overall shapes are correctly reconstructed.  

>The result of this process is that the auto-encoder outputs the same image as it receives as input, while passing through a **latent space** of smaller dimension: here 15.
>This may not seem very useful, but as we've separated the auto-encoder into an encoder on the one hand and a decoder on the other, we can use them separately.  
>For example, keeping only the encoder, we can represent a one-digit image made up of 784 pixels with a vector of 15 values.

In [ ]:
x_test_latent = encoder.predict(X_test)

We've just designed a "Generative" model that can be used in a number of applications, for example:
* Dimension reduction: as mentioned above, we can use the Encoder function (Encoder part of the network) to reduce the dimension of input data. For example, an image is given as input to the Encoder, which produces, according to the previous example, a vector containing 15 values (between 0 and 1).
* Generating new images: we can also use the Decoder part of the network and feed it with vectors (15 values in the previous example) generated at random. By doing so, the Decoder will generate new images not contained in the Dataset.
* Noise reduction (Denoising): we can also train the AutoEncoder with noisy input and denoised output images, enabling us to design a Denoising AutoEncoder. This is shown in the following image:
![denoising-autoencoder-architecture.png](attachment:denoising-autoencoder-architecture.png)
By the way, you'll see this application in detail in the next WOrkshop!

Finally, and to conclude on AEs, these networks are very sensitive to the phenomenon of overlearning. To avoid this phenomenon, an improved version of this concept is used today under the name: "Variational AutoEncoder". The general concept remains the same: we create an Encoder and a Decoder. However, instead of using scalar functions, we'll define probability functions to encode and decode the image.

### Variational auto-encoder

In this second part, we'll implement a variational autoencoder. Let's start by defining the specific layer we'll use to sample values from the Gaussian distribution defined by the means and standard deviations of the latent space (z_mean, z_logvar).
![vae-gaussian.png](attachment:vae-gaussian.png)

In [ ]:
# Coding the specific sampling layer as a Keras Layer object
class Sampling(layers.Layer):

    def call(self, inputs):
        z_mean, z_logvar = inputs

        nbatch = K.shape(z_mean)[0]
        ndim = K.shape(z_mean)[1]

        std = K.exp(z_logvar)
        eps = K.random_normal(shape=(nbatch,ndim), mean=0., stddev=0.1)

        z = z_mean + eps * std

        return z


Now let's write the separate encoder and decoder architectures. Please note: no activation function for averaging and logvar...

In [ ]:
encoder_inputs = keras.Input(shape=(784,))
hidden1 = layers.Dense(200, activation="relu")(encoder_inputs)

z_mean = layers.Dense(10)(hidden1)
z_logvar = layers.Dense(10)(hidden1)

# Sampling
z = Sampling()([z_mean, z_logvar])

encoder = keras.Model(encoder_inputs, z, name="encoder")


In [ ]:
from keras.utils import plot_model
plot_model(encoder, to_file='encoder.png', show_shapes=True)

In [ ]:
# Decoder architecture
decoder_inputs = keras.Input(shape=(10,))
decoder_hidden = layers.Dense(200, activation="relu")(decoder_inputs)
decoder_outputs = layers.Dense(784, activation="sigmoid")(decoder_hidden)

decoder = keras.Model(decoder_inputs, decoder_outputs, name="decoder")

In [ ]:
from keras.utils import plot_model
plot_model(decoder, to_file='decoder.png', show_shapes=True)

Now we combine them to build the complete automatic encoder.

In [ ]:
# Combining architectures
outputs = decoder(z)
vae = keras.Model(encoder_inputs, outputs, name="vae")

In [ ]:
vae.summary()

We now need to define the learning loss function. As a reconstruction loss, we can always use the binary cross entropy per pixel, summed over the image. In the case of VAE, there is an additional term to the loss: the [Kullback-Leibler divergence](https://fr.wikipedia.org/wiki/Divergence_de_Kullback-Leibler). Write this new term, using Keras backend functions: K.square, K.exp, K.sum...  

In [ ]:
# Loss function definition
reconstruction_loss = #PLEASE COMPLETE

kl_loss = #PLEASE COMPLETE

vae_loss = K.mean(reconstruction_loss + kl_loss)


In [ ]:
# Compiling the model
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [ ]:
# Fitting the model
history = vae.fit(X_train, X_train,
          epochs=30,
          batch_size=128,
          shuffle=True,
          validation_data=(X_test, X_test))

In [ ]:
# Visualizing the training and validation losses
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

We seem to have achieved a fairly reasonable training process! Let's have a look at the quality of the reconstructed images.

In [ ]:
n = 10
plt.figure(figsize=(20, 4))

for i in range(n):
    # Original images
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28), cmap="gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Reconstructed images
    ax = plt.subplot(2, n, i + 1 + n)
    encoded_img = encoder.predict(np.array([X_test[i]]).reshape(1,784))
    decoded_img = decoder.predict(encoded_img).reshape(28, 28)
    plt.imshow(decoded_img, cmap="gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

We can see that some details are lost, but the overall shapes are correctly reconstructed.

As a VAE uses a generative model, we can use the decoder to construct fake images, and we can see that the latent space is continuous! Let's sample a few random images in a given range of latent space values.

In [ ]:
n = 15  # figure with 15x15 generated images
figure = np.zeros((28 * n, 28 * n))

# We sample n images within [-15, 15] standard deviations, around 0 mean
# Vincent:Comment définir les valeurs pour générer les images.
grid_x = np.linspace(-15, 15, n)
grid_y = np.linspace(-15, 15, n)

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):

        # We sample the latent space over the 2 first neurons, feel free to change that to other pairs!
        z_sample = np.array([[xi, yi, 0, 0, 0, 0, 0, 0, 0, 0]])
        x_decoded = decoder.predict(z_sample)

        digit = x_decoded[0].reshape(28, 28)
        figure[i * 28: (i + 1) * 28,
               j * 28: (j + 1) * 28] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap="gray")
plt.show()

In [ ]:
x_test_latent = encoder.predict(X_test)